In [ ]:
# 분야 2 | 유선 네트워크 장비의 신속한 조치를 위한 경보 유형 분류

# 아래의 코드는 다음과 같은 순서로 구성되어 있습니다.
# 1. 데이터 로드 및 전처리
# 2. 모델 학습 및 예측
# 3. 결과 제출
# 본 코드는 결과물 제출까지의 이해를 돕기위한 코드로, 반드시 이 구조를 따를 필요는 없습니다.
# 데이터 전처리, 모델의 구성 등에서 다양한 시도를 하시되, 올바른 채점을 위한 최종 결과물의 형태에 유의하시기 바랍니다.

In [2]:
# 1. 데이터 로드
import numpy as np
import pandas as pd

def ppr_data(q2_train, q2_test):
    # TODO: 데이터 전처리 코드 구현 ---------- #
    # slotna 변수 생성
    q2_train['slotna'] = q2_train['slot'].isna().astype(int)
    q2_test['slotna'] = q2_test['slot'].isna().astype(int)

    # 메시지 전처리
    for i in range(len(q2_train)):
      s = q2_train['alarmmsg_original'].iloc[i]
      s = s.lower()
      s = s.replace('(', '').replace(')', '') # 괄호 없애기
      s = s.replace('-', ' ').replace('_', ' ') # '-', '_' 구분자 공백으로 바꾸기
      s = s.split() # 공백 기준으로 split
      for _ in s: # 조사나 의미 없는 단어 삭제
          if 'of' in s:
              s.remove('of')
          if 'los' in s:
              s.remove('los')
          if 'loss' in s:
              s.remove('loss')
          if 'by' in s:
              s.remove('by')
          if 'with' in s:
              s.remove('with')
          if 'fail' in s:
              s.remove('fail')
          if 'error' in s:
              s.remove('error')
          if 'err' in s:
              s.remove('err')
      q2_train['alarmmsg_original'].iloc[i] = s

    for i in range(len(q2_test)):
      if q2_test['root_cause_domain'].iloc[i,] == 'C':
          input_string = q2_test['alarmmsg_original'].iloc[i,]
          uppercase_letters = [char for char in input_string if char.isupper()]
          result = ''.join(uppercase_letters)

          # result 전처리
          if result == 'LOS' or result == 'LC':
              result = ''

          # 원래 메시지 전처리
          input_string = input_string.lower()
          input_string = input_string.replace('(', '').replace(')', '') # 괄호 없애기
          input_string = input_string.replace('-',' ').replace('_',' ').replace(':', '') # ':' 없애기
          input_string = input_string.split() # 공백 기준으로 split
          for _ in input_string: # 조사나 의미 없는 단어 삭제
              if 'of' in input_string:
                  input_string.remove('of')
              if 'los' in input_string:
                  input_string.remove('los')
              if 'loss' in input_string:
                  input_string.remove('loss')
              if 'by' in input_string:
                  input_string.remove('by')
              if 'with' in input_string:
                  input_string.remove('with')
              if 'fail' in input_string:
                  input_string.remove('fail')
              if 'error' in input_string:
                  input_string.remove('error')
              if 'err' in input_string:
                  input_string.remove('err')
              if 'failure' in input_string:
                  input_string.remove('failure')
          if result:
              q2_test['alarmmsg_original'].iloc[i,] = [input_string, [result.lower()]]
              q2_test['alarmmsg_original'].iloc[i,] = sum(q2_test['alarmmsg_original'].iloc[i,], [])
          else:
              q2_test['alarmmsg_original'].iloc[i,] = input_string

      else:
          input_string1 = q2_test['alarmmsg_original'].iloc[i,]

          # 원래 메시지 전처리
          input_string1 = input_string1.lower()
          input_string1 = input_string1.replace('(', '').replace(')', '') # 괄호 없애기
          input_string1 = input_string1.replace('-',' ').replace('_',' ').replace(':', '') # ':' 없애기
          input_string1 = input_string1.split() # 공백 기준으로 split
          for _ in input_string: # 조사나 의미 없는 단어 삭제
              if 'of' in input_string1:
                  input_string1.remove('of')
              if 'los' in input_string1:
                  input_string1.remove('los')
              if 'loss' in input_string1:
                  input_string1.remove('loss')
              if 'by' in input_string1:
                  input_string1.remove('by')
              if 'with' in input_string1:
                  input_string1.remove('with')
              if 'fail' in input_string1:
                  input_string1.remove('fail')
              if 'error' in input_string1:
                  input_string1.remove('error')
              if 'err' in input_string1:
                  input_string1.remove('err')
              if 'failure' in input_string1:
                  input_string1.remove('failure')
              if 'ether' in input_string1:
                  input_string1 = ['eth']

          q2_test['alarmmsg_original'].iloc[i,] = input_string1

    # 필요 없는 변수 drop
    q2_train.drop(labels = ['alarmno', 'slot', 'port', 'sva', 'root_cause_domain'], axis = 1, inplace = True)
    q2_test.drop(labels = ['alarmno', 'slot', 'port', 'sva', 'root_cause_domain'], axis = 1, inplace = True)

    # 열 순서 정렬
    col = q2_train.columns.to_numpy()
    col = col[[0, 1, 2, 3, 4, 5, 6, 8, 7]]
    q2_train = q2_train[col]

    # time 변수 datetime으로 타입 변경
    q2_train['alarmtime'] = pd.to_datetime(q2_train['alarmtime'])
    q2_test['alarmtime'] = pd.to_datetime(q2_test['alarmtime'])

    # train, test 데이터 합치기
    q2_train['train'] = 1
    q2_test['train'] = 0
    col = q2_train.columns.to_numpy()
    col = col[[0, 1, 2, 3, 4, 5, 6, 7, 9, 8]]
    q2_train = q2_train[col]
    data_total = pd.concat([q2_train, q2_test])

    # alarmtime 변수로 지속시간 변수 추가 및 time변수 삭제
    data = data_total.groupby('ticketno')['alarmtime'].agg(**{'min_time':'min', 'max_time':'max'}).reset_index()
    data['duration'] = data['max_time'] - data['min_time']
    for i in range(len(data['duration'])):
      data['duration'][i] = data['duration'][i].seconds / 60

    # 전표별 level 당 개수 변수 생성
    level = data_total.groupby(['ticketno','alarmlevel']).size().unstack(fill_value = 0).reset_index()
    data['level3'] = level[3]
    data['level4'] = level[4]
    data['level5'] = level[5]
    data['level7'] = level[7]
    data.drop(labels=['min_time', 'max_time'], axis = 1, inplace = True)

    # 전표별 alarmmsg 이어붙이기
    message = data_total.groupby('ticketno')['alarmmsg_original'].apply(list).reset_index()
    for i in range(len(message)):
      message['alarmmsg_original'].iloc[i] = sum(message['alarmmsg_original'].iloc[i],[])

    data = pd.merge(data, message)
    data = pd.merge(data, data_total.groupby('ticketno')['train'].agg(**{'train':max}).reset_index())
    data = pd.merge(data, data_total.groupby('ticketno')['root_cause_type'].agg(**{'type':max}).reset_index())
    data = pd.merge(data, data_total.groupby('ticketno')['slotna'].agg(**{'slotna': max}).reset_index())

    col = data.columns.to_numpy()
    col = col[[0, 1, 2, 3, 4, 5, 6, 7, 9, 8]]
    data = data[col]

    # train, test 분리
    data_train = data[data['train'] == 1]
    data_test = data[data['train'] == 0]
    data_train.drop(labels = ['train'], axis = 1, inplace = True)
    data_test.drop(labels = ['train'], axis = 1, inplace = True)

    # x, y 변수 분리
    x_train_df = data_train.iloc[:, :-1]
    y_train_df = data_train['type']
    x_test_df = data_test.iloc[:, :-1]

    # ------------------------------------- #
    return x_train_df, y_train_df, x_test_df


train_df = pd.read_csv("/content/Q2_train.csv")
test_df = pd.read_csv("/content/Q2_test.csv")

x_train_df, y_train_df, x_test_df = ppr_data(train_df, test_df)

<ipython-input-2-d52e3bac14fb>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q2_train['alarmmsg_original'].iloc[i] = s
<ipython-input-2-d52e3bac14fb>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q2_test['alarmmsg_original'].iloc[i,] = input_string
<ipython-input-2-d52e3bac14fb>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q2_test['alarmmsg_original'].iloc[i,] = input_string1
<ipython-input-2-d52e3bac14fb>:72: SettingWith

In [5]:
# 2. 모델 학습 및 예측
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

class MyModel:
    def __init__(self) -> None:
        self.model = None

    def train(self, x_train, y_train):
        # TODO: 모델 학습 코드 구현 ---------- #
        # TfidfVertorizer로 메시지 수치화 후 Multinomial naive bayes 모델링

        # Create transformers
        message_vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)
        # additional_scaler = MinMaxScaler()

        # Define column indices for the ColumnTransformer
        message_col_idx = 'alarmmsg_original'
        additional_cols = ['duration', 'level3', 'level4', 'level5', 'level7', 'slotna']

        # Create the ColumnTransformer
        preprocessor = ColumnTransformer(
            transformers=[
                ('message', message_vectorizer, message_col_idx),
            ],
            remainder='passthrough'
        )
        # Create the Xgboost model
        xgb_model = XGBClassifier()

        # Create a pipeline with the ColumnTransformer and the classifier
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', xgb_model)
        ])

        # Encode class labels to integers
        global label_encoder
        label_encoder = LabelEncoder()
        y_train = label_encoder.fit_transform(y_train)

        x_train = x_train[['alarmmsg_original'] + additional_cols]

        # Train the pipeline
        self.model = pipeline.fit(x_train, y_train)


        # --------------------------------- #
        pass

    def predict(self, x_test):
        # 1. ticketno 컬럼은 입력받은 값으로 채우고,
        # 2. pred 컬럼은 모두 'LinkCut' 값으로 채운 데이터프레임 생성
        pred = self.model.predict(x_test)
        # Convert predictions back to original class labels
        predicted_class_labels = label_encoder.inverse_transform(pred)
        pred_df = pd.DataFrame({'ticketno': x_test['ticketno'].values, 'root_cause_type': predicted_class_labels})
        return pred_df

model = MyModel()
model.train(x_train_df, y_train_df)
y_pred = model.predict(x_test_df) # 0.82158

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
list(y_pred['root_cause_type']).count('LinkCut')

2545

In [8]:
# 3. 결과 제출
# 본 코드는 제출되는 파일의 형태에 대한 가이드로, 반드시 아래 구조를 따를 필요 없이 자유롭게 코드를 작성해도 무방합니다.
# 제출 포맷에 대해선 data/Q2_label_sample.csv를 참조하세요.
#
# 분야 2의 경우, 전표(ticket) 하나에 하나의 근원장애(root_cause_type)을 매칭해야 합니다.
#   주의: 경보(alarm) 개수와 전표(ticket) 개수는 다르며, 예측할 대상은 전표입니다.
#   주의: ticketno 컬럼 기준으로 오름차순 정렬이 필요합니다.
# 분야 2의 제출 파일은 2개 컬럼 [ticketno, root_cause_type]을 가져야 합니다.

def submitResult(pred):
    try:
        label = pd.read_csv('Q2_label_sample.csv')
        # ticketno 순서와 개수가 일치하는지 확인
        if (label['ticketno'] == pred['ticketno']).all():
            print("Check: ticketno 순서와 샘플 수가 일치합니다.")
        else:
            print("Warning: 테스트 세트와 모델 예측의 ticketno가 일치하지 않습니다.")
            return

        pred.to_csv('Q2_submitResult.csv', index=False)
        print("Done : Q2_submitResult.csv 파일로 저장되었습니다.")
    except Exception as e:
        # 예외가 발생한 경우 오류 메시지 출력
        print("Error:", e)

submitResult(y_pred)

Check: ticketno 순서와 샘플 수가 일치합니다.
Done : Q2_submitResult.csv 파일로 저장되었습니다.
